In [1]:
"""
Created on Fri Jul 18 18:20:02 2025

@author: huzefa
"""

'\nCreated on Fri Jul 18 18:20:02 2025\n\n@author: huzefa\n'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import statsmodels.api as sm

In [3]:
cars_omit_data=pd.read_csv('../Linear-Regression/omitted_data_cars_sampled.csv')
cars_omit_data=cars_omit_data.drop('Unnamed: 0',axis=1)

In [4]:
x1 = cars_omit_data.drop(['price','model','brand'], axis='columns', inplace=False)
x1=pd.get_dummies(x1,drop_first=True) 
y1= cars_omit_data.filter(['price'],axis=1)
# Transforming prices to log
y2=np.log(y1)

In [5]:
X_train, X_test, y_train_log,y_test_log = train_test_split(x1, y2, test_size=0.3, random_state = 3)
print(X_train.shape, X_test.shape, y_train_log.shape, y_test_log.shape)

(23018, 18) (9866, 18) (23018, 1) (9866, 1)


In [6]:
def rmse_log(test_y,predicted_y):
    t1=np.exp(test_y)
    t2=np.exp(predicted_y)
    rmse_test=np.sqrt(mean_squared_error(t1,t2))
    #for base rmse
    base_pred = np.repeat(np.mean(t1), len(t1))
    rmse_base = np.sqrt(mean_squared_error(t1, base_pred))
    values={'RMSE-test from model':rmse_test,'Base RMSE':rmse_base}
    return values

In [7]:
rf = RandomForestRegressor(n_estimators = 220,max_depth=87)

In [8]:
# Model
model_rf1=rf.fit(X_train,y_train_log)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [9]:
# Predicting model on test and train set
cars_predictions_rf1_test = rf.predict(X_test)

In [10]:
# RMSE
rmse_log(y_test_log,cars_predictions_rf1_test)

{'RMSE-test from model': 3908.8507485464643, 'Base RMSE': 8189.787602929656}

In [11]:
# Rsquared
model_rf1.score(X_train,y_train_log)

0.9646680535895141

In [12]:
## Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 600,num = 15)]
print(n_estimators)

[10, 52, 94, 136, 178, 220, 262, 305, 347, 389, 431, 473, 515, 557, 600]


In [13]:
## Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]

In [14]:
## Minimum number of samples required to split a node
min_samples_split = np.arange(100,1100,100)

In [15]:
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

In [16]:
print(random_grid)

{'n_estimators': [10, 52, 94, 136, 178, 220, 262, 305, 347, 389, 431, 473, 515, 557, 600], 'max_depth': [10, 21, 32, 43, 54, 65, 76, 87, 98, 110], 'min_samples_split': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000])}


In [17]:
rf_for_tuning = RandomForestRegressor()

In [18]:
rf_random = RandomizedSearchCV(estimator = rf_for_tuning, 
                            param_distributions = random_grid, 
                            n_iter = 100,cv = 3, verbose=2, random_state=1)

In [19]:
## Fit the random search model
rf_random.fit(X_train,y_train_log)
print(rf_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=10, min_samples_split=700, n_estimators=52; total time=   0.3s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=10, min_samples_split=700, n_estimators=52; total time=   0.3s


...

[CV] END max_depth=87, min_samples_split=200, n_estimators=557; total time=   4.8s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=87, min_samples_split=200, n_estimators=557; total time=   4.7s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'n_estimators': 389, 'min_samples_split': 100, 'max_depth': 32}


In [20]:
## finding the best model
rf_model_best = rf_random.best_estimator_
print(rf_model_best)

RandomForestRegressor(max_depth=32, min_samples_split=100, n_estimators=389)


In [21]:
# predicting with the test data on best model
predictions_best = rf_model_best.predict(X_test)
predictions_best_train=rf_model_best.predict(X_train)